In [2]:
import numpy as np
import pandas as pd
import subprocess as sp
import os
from scipy.stats import zscore
import seaborn as sb
import matplotlib.cm as cm
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
from collections import Counter


plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 1
plt.rcParams["axes.facecolor"] = 'white'

This file reads the cassette exons events found by running rMATS, and uses them as guides to find cassette exons in single cells using the SJ.out.tab files from STAR.

In [3]:
SJ_dir = '/mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/'

This NMD file is used for purposes of naming the NMD exons only. It can be ignored.

In [4]:
nmd_events = pd.read_table('~/Network/Mouse/Events/NMD_tables/protein_coding/nmd_se.tab', sep = '\t', index_col=None)
nmd_events.start = (nmd_events.start - 1)

/mnt/lareaulab/cfbuenabadn/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


In [5]:
rmats_file = '../../HubbardRNASeq/splicing/rMATS/output_2//{time}/MATS_output/SE.MATS.ReadsOnTargetAndJunctionCounts.txt'
awkCommand = """awk -F'\\t' '$4=="{chrom}" && $5=="{strand}" && $9=={ie1_st} && $6=={ie1_end}"""
awkCommand += """ && $7=={ie2_st} && $10=={ie2_end}' {rmats_file}"""
nmd_ase = {}
event_id = {}

for ase in nmd_events.event.unique():
    print(ase)
    SJ_row = []
    event_slice = nmd_events.loc[nmd_events.event == ase]
    
    chrom = list(event_slice.chrom)[0]
    strand = list(event_slice.strand)[0]
    ie1_start = list(event_slice.start)[0]
    ie1_end = list(event_slice.end)[0]
    ie2_start = list(event_slice.start)[1]
    ie2_end = list(event_slice.end)[1]
    se_start = list(event_slice.start)[2]
    se_end = list(event_slice.end)[2]
    
    
    nmd_ase.update({ase:{'DN8':[], 'DN4':[], 'D0':[], 'D1':[], 'D7':[], 'D16':[], 'D21':[], 'D28':[], 'qval':[]}})
    min_qval = 1
    for time in ['DN4', 'D0', 'D1', 'D7', 'D16', 'D21', 'D28']:
        cmd = awkCommand.format(chrom=chrom, strand=strand, ie1_st=ie1_start, ie1_end=ie1_end, 
                                ie2_st=ie2_start, ie2_end=ie2_end, rmats_file=rmats_file.format(time=time))
        result = sp.check_output(cmd, shell=True).decode()
        token = False
        if result == '':
            nmd_ase[ase][time].extend([0,0,0])
        else:
            D0_psi = []
            for x in result.split('\t')[20].split(','):
                if x == 'NA':
                    D0_psi.append(0)
                else:
                    D0_psi.append(float(x))
            psi = []
            for x in result.split('\t')[21].split(','):
                if x == 'NA':
                    psi.append(0)
                else:
                    psi.append(float(x))
            nmd_ase[ase][time].extend(psi)
            token = True
            min_qval = np.min([min_qval, float(result.split('\t')[19])])
            rows = result.split('\t')
            ase_id = [rows[3], rows[8], rows[5], rows[6], rows[9], rows[4]]
            ase_id = [str(x) for x in ase_id]
            ase_id = ':'.join(ase_id)
            if ase_id not in event_id.keys():
                event_id.update({ase_id:ase})
    if token:
        nmd_ase[ase]['D0'].extend(D0_psi)
    else:
        nmd_ase[ase]['D0'].extend([0,0,0])
    nmd_ase[ase]['qval'].append(min_qval)
            

St18_nmdSE_1
Pcmtd1_nmdSE_1
Stau2_nmdSE_1
Ube2w_nmdSE_1
Prss39_nmdSE_1
Lman2l_nmdSE_1
Tmem131_nmdSE_1
Rev1_nmdSE_1
Chst10_nmdSE_1
Mfsd9_nmdSE_1
Uxs1_nmdSE_1
Kdelc1_nmdSE_1
Stat1_nmdSE_1
Osgepl1_nmdSE_1
Asnsd1_nmdSE_1
Sf3b1_nmdSE_1
Rftn2_nmdSE_1
Nif3l1_nmdSE_1
Fam126b_nmdSE_1
Stradb_nmdSE_1
Nop58_nmdSE_1
Fam117b_nmdSE_1
Carf_nmdSE_1
Carf_nmdSE_2
Ndufs1_nmdSE_1
Eef1b2_nmdSE_1
Creb1_nmdSE_1
Creb1_nmdSE_2
Pikfyve_nmdSE_1
Dnajb2_nmdSE_1
Cul3_nmdSE_1
Dis3l2_nmdSE_1
Usp40_nmdSE_1
Thap4_nmdSE_1
Slco6d1_nmdSE_1
2310035C23Rik_nmdSE_1
Zcchc2_nmdSE_1
Serpinb13_nmdSE_1
Cdh7_nmdSE_1
Tmem183a_nmdSE_1
Cyb5r1_nmdSE_1
Mgat4e_nmdSE_1
Ipo9_nmdSE_1
Ipo9_nmdSE_2
Kif21b_nmdSE_1
Crb1_nmdSE_1
Uchl5_nmdSE_1
Brinp3_nmdSE_1
Fam129a_nmdSE_1
1700025G04Rik_nmdSE_1
Rgl1_nmdSE_1
Nmnat2_nmdSE_1
Dhx9_nmdSE_1
Rgsl1_nmdSE_1
Tor1aip1_nmdSE_1
Tor3a_nmdSE_1
Ralgps2_nmdSE_1
Astn1_nmdSE_1
Rfwd2_nmdSE_1
Cenpl_nmdSE_1
Gorab_nmdSE_1
Mettl11b_nmdSE_1
Sft2d2_nmdSE_1
Creg1_nmdSE_1
Tada1_nmdSE_1
Uck2_nmdSE_1
Ncstn_nmdSE_1
Copa_nmdSE_

Ints4_nmdSE_1
Aamdc_nmdSE_1
Aamdc_nmdSE_2
Rsf1_nmdSE_1
Rsf1_nmdSE_2
Rsf1_nmdSE_3
Acer3_nmdSE_1
Prkrir_nmdSE_1
Uvrag_nmdSE_1
Uvrag_nmdSE_2
Uvrag_nmdSE_3
Dgat2_nmdSE_1
Spcs2_nmdSE_1
C2cd3_nmdSE_1
Pde2a_nmdSE_1
Clpb_nmdSE_1
Pgap2_nmdSE_1
Pgap2_nmdSE_2
Rrm1_nmdSE_1
Parva_nmdSE_1
Nucb2_nmdSE_1
Pdilt_nmdSE_1
Acsm5_nmdSE_1
BC030336_nmdSE_1
Ubfd1_nmdSE_1
Dctn5_nmdSE_1
Il4ra_nmdSE_1
Gtf3c1_nmdSE_1
Xpo6_nmdSE_1
Rabep2_nmdSE_1
Rabep2_nmdSE_2
Sh2b1_nmdSE_1
Cln3_nmdSE_1
Zfp688_nmdSE_1
Bcl7c_nmdSE_1
Tial1_nmdSE_1
Tial1_nmdSE_2
Tial1_nmdSE_3
Fgfr2_nmdSE_1
Nsmce4a_nmdSE_1
Plekha1_nmdSE_1
Bub3_nmdSE_1
Lhpp_nmdSE_1
Glrx3_nmdSE_1
Ppp2r2d_nmdSE_1
Cfap46_nmdSE_1
Sirt3_nmdSE_1
Pkp3_nmdSE_1
Ptdss2_nmdSE_1
Deaf1_nmdSE_1
Deaf1_nmdSE_2
Deaf1_nmdSE_3
Pnpla2_nmdSE_1
Chid1_nmdSE_1
Tspan32_nmdSE_1
Trpm5_nmdSE_1
Gm498_nmdSE_1
Gm498_nmdSE_2
Tpcn2_nmdSE_1
Arglu1_nmdSE_1
Col4a1_nmdSE_1
Cars2_nmdSE_1
Ankrd10_nmdSE_1
Ankrd10_nmdSE_2
Arhgef7_nmdSE_1
Tex29_nmdSE_1
Spaca7_nmdSE_1
Pcid2_nmdSE_1
Cul4a_nmdSE_1
Grtp1_nmdSE_1
Tf

In [6]:
rmats_file = '../../HubbardRNASeq/splicing/rMATS/output_2/D28/MATS_output/SE.MATS.ReadsOnTargetAndJunctionCounts.txt'
SF_list = list(pd.read_table('~/Network/Mouse/Events/mouse_SF_RBP.txt', sep = '\t').mgi_symbol)
table = pd.read_table(rmats_file, sep = '\t', index_col=None)

/mnt/lareaulab/cfbuenabadn/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead.
  
/mnt/lareaulab/cfbuenabadn/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
rmats_file = '../../HubbardRNASeq/splicing/rMATS/output_2/{day}/MATS_output/SE.MATS.ReadsOnTargetAndJunctionCounts.txt'
SF_events_temp = []
events = {}

for day in ['DN4', 'D0', 'D1', 'D7', 'D16', 'D21', 'D28']:
    
    table = pd.read_table(rmats_file.format(day=day), sep = '\t', index_col=None)
    
    print(rmats_file.format(day=day))
    for idx, rows in table.iterrows():
        
        ase_id = [rows[3], rows[8], rows[5], rows[6], rows[9], rows[4]]
        ase_id = [str(x) for x in ase_id]
        ase_id = ':'.join(ase_id)
        gene = rows[2].strip('"')
        
        #####

        if ase_id not in event_id.keys():
            id_num = np.sum([event_id[x].split('_')[0]==gene for x in event_id.keys()])
            ase = gene + '_' + str(id_num+1)
            event_id.update({ase_id:ase})
        
    
        if ase_id not in events.keys():
            events.update({ase_id:{'D0':rows[20], day:rows[21], 'fdr':float(rows[19])}})
        else:
            min_fdr = np.min([events[ase_id]['fdr'], rows[19]])
            events[ase_id].update({day:rows[21], 'fdr':min_fdr})
        
        if ase_id not in SF_events_temp and gene in SF_list:
            SF_events_temp.append(ase_id)
                          

/mnt/lareaulab/cfbuenabadn/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: read_table is deprecated, use read_csv instead.
  import sys


../../HubbardRNASeq/splicing/rMATS/output_2/DN4/MATS_output/SE.MATS.ReadsOnTargetAndJunctionCounts.txt
../../HubbardRNASeq/splicing/rMATS/output_2/D0/MATS_output/SE.MATS.ReadsOnTargetAndJunctionCounts.txt
../../HubbardRNASeq/splicing/rMATS/output_2/D1/MATS_output/SE.MATS.ReadsOnTargetAndJunctionCounts.txt
../../HubbardRNASeq/splicing/rMATS/output_2/D7/MATS_output/SE.MATS.ReadsOnTargetAndJunctionCounts.txt
../../HubbardRNASeq/splicing/rMATS/output_2/D16/MATS_output/SE.MATS.ReadsOnTargetAndJunctionCounts.txt
../../HubbardRNASeq/splicing/rMATS/output_2/D21/MATS_output/SE.MATS.ReadsOnTargetAndJunctionCounts.txt
../../HubbardRNASeq/splicing/rMATS/output_2/D28/MATS_output/SE.MATS.ReadsOnTargetAndJunctionCounts.txt


In [8]:
with open('ase_SJ.txt', 'a') as ase_file:
    
    for event in events.keys():
        name = event_id[event]
        partes = event.split(':')
        SJs = [name + '\t' + name + '_I1' + '\t' + partes[0] + '\t' + str(int(partes[1])+1) + '\t' + partes[2] + '\n',
               name + '\t' + name + '_I2' + '\t' + partes[0] + '\t' + str(int(partes[3])+1) + '\t' + partes[4] + '\n',
               name + '\t' + name + '_SE' + '\t' + partes[0] + '\t' + str(int(partes[1])+1) + '\t' + partes[4] + '\n']
        ase_file.writelines(SJs)

In [9]:
cells = [x.split('.')[0] for x in os.listdir(SJ_dir)]

In [10]:
SJ_counts = {}
SJ_converter = {}

with open('ase_SJ.txt', 'r') as ase_file:
    for row in ase_file:
        sj = row.rstrip().split('\t')
        key = sj[2] + ':' + sj[3] + ':' + sj[4]
        if key in SJ_converter.keys():
            SJ_converter[key].append(sj[1])
        else:
            SJ_converter.update({key:[sj[1]]})
        SJ_counts.update({sj[1]:[0]*len(cells)})


In [11]:
counter = 0
for cell in cells:
    SJ_table = SJ_dir + cell + '.SJ.out.tab'
    with open(SJ_table, 'r') as SJ_file:
        for row in SJ_file:
            sj = row.rstrip().split('\t')
            row_key = sj[0] + ':' + sj[1] + ':' + sj[2]
            if row_key in SJ_converter.keys():
                ase_names = SJ_converter[row_key]
                for ase in ase_names:
                    SJ_counts[ase][counter] = int(sj[6]) + int(sj[7])
                
            row_key = sj[0] + ':' + sj[2] + ':' + sj[1]
            if row_key in SJ_converter.keys():
                ase_names = SJ_converter[row_key]
                for ase in ase_names:
                    SJ_counts[ase][counter] = int(sj[6]) + int(sj[7])
        
    print(cell)
    counter += 1

SRR3211941
SRR2727160
SRR2727334
SRR2727190
SRR2727113
SRR3211932
SRR2727538
SRR2727347
SRR2727322
SRR2727176
SRR2727186
SRR2727351
SRR3211924
SRR2727105
SRR2727318
SRR2727567
SRR2727514
SRR2727597
SRR2727571
SRR3211908
SRR2727129
SRR2727502
SRR2727581
SRR2727497
SRR2727414
SRR2727218
SRR2727467
SRR2727481
SRR2727402
SRR2727471
SRR2727438
SRR2727247
SRR2727090
SRR2727234
SRR2727060
SRR2727251
SRR2727086
SRR2727076
SRR2727609
SRR2727222
SRR2727425
SRR2727602
SRR2727229
SRR2727456
SRR2727433
SRR2727440
SRR2727614
SRR2727409
SRR2727276
SRR2727286
SRR2727205
SRR2727051
SRR2727260
SRR2727290
SRR2727047
SRR2727213
SRR2727151
SRR2727305
SRR2727386
SRR2727122
SRR2727509
SRR2727376
SRR2727313
SRR2727147
SRR2727390
SRR2727360
SRR3211915
SRR2727134
SRR2727329
SRR2727556
SRR2727525
SRR2727540
SRR3211939
SRR2727118
SRR2727533
SRR2727613
SRR2727238
SRR2727447
SRR2727434
SRR2727451
SRR2727605
SRR2727422
SRR2727214
SRR2727297
SRR2727418
SRR2727267
SRR2727056
SRR2727202
SRR2727281
SRR2727271
SRR2727133

In [12]:
SJ_counts_table = pd.DataFrame.from_dict(SJ_counts)
SJ_counts_table.index = cells

In [13]:
SJ_counts_table.shape

(617, 162945)

In [14]:
#SJ_counts_table.T.to_csv('SJ_counts.tab', sep='\t', index=True, header=True)

In [15]:
SJ_dir = '/mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/'
grep_nmd1_cmd = 'grep chr4 ' + SJ_dir + '{cell}.SJ.out.tab | grep 135858472 | grep 135858841'
grep_nmd2_cmd = 'grep chr4 ' + SJ_dir + '{cell}.SJ.out.tab | grep 135859202 | grep 135861770'
grep_pc_cmd = 'grep chr4 ' + SJ_dir + '{cell}.SJ.out.tab | grep 135858410 | grep 135861773'

nmd1 = []
nmd2 = []
pc = []

SJ_list = SJ_counts_table.index

for cell in SJ_list:
    try:
        cmd = grep_nmd1_cmd.format(cell=cell)
        print(cmd)
        row = sp.check_output(cmd, shell=True).decode()
        print(row)
        sj = row.rstrip().split('\t')
        nmd1.append(int(sj[6]) + int(sj[7]))
    except:
        nmd1.append(0)
        
    try:
        cmd = grep_nmd2_cmd.format(cell=cell)
        row = sp.check_output(cmd, shell=True).decode()
        sj = row.rstrip().split('\t')
        nmd2.append(int(sj[6]) + int(sj[7]))
    except:
        nmd2.append(0)
        
    try:
        cmd = grep_pc_cmd.format(cell=cell)
        row = sp.check_output(cmd, shell=True).decode()
        sj = row.rstrip().split('\t')
        pc.append(int(sj[6]) + int(sj[7]))
    except:
        pc.append(0)
        
        

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR3211941.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	14	0	18

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727160.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	2	0	18

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727334.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	14	0	20

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727190.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	9	0	20

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727113.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	3	0	20

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR3211932.SJ.out.tab | grep 135858472 | gr

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727286.SJ.out.tab | grep 135858472 | grep 135858841
grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727205.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	7	0	20

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727051.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	10	0	20

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727260.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	16	0	20

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727290.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	15	0	20

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727047.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727125.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	8	0	20

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727381.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	7	0	20

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727302.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	31	0	20

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727156.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	6	0	21

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727534.SJ.out.tab | grep 135858472 | grep 135858841
grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727338.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727097.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	4	0	14

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727240.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	1	0	20

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727400.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	4	0	20

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727483.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	1	0	6

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727473.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	10	0	20

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727058.SJ.out.tab | grep 135858472 | grep

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727384.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	5	0	20

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727153.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	9	0	18

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727578.SJ.out.tab | grep 135858472 | grep 135858841
grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727307.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	10	0	20

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727120.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	10	0	21

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727374.SJ.out.tab | grep 135858472 | grep 135858841
grep chr4 /mnt/lareaulab/c

...

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727582.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	7	0	20

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727501.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	2	0	10

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727564.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	2	0	18

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727398.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	7	0	20

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727594.SJ.out.tab | grep 135858472 | grep 135858841
grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727517.SJ.out.tab | grep 135858472 | grep 135858841
grep chr4 /mnt/lareaulab/cfb

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727443.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	4	0	20

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727068.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	2	0	18

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727617.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	15	0	20

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727426.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	6	0	18

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727259.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	6	0	20

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727601.SJ.out.tab | grep 135858472 | gre

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727419.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	4	0	19

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727057.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	6	0	18

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727203.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	1	0	18

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727280.SJ.out.tab | grep 135858472 | grep 135858841
grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727270.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	2	0	10

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727612.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727332.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	10	0	20

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727166.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	1	0	19

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727246.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	1	0	18

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727439.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	10	0	20

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727091.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	17	0	20

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727235.SJ.out.tab | grep 135858472 | g

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727304.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	1	0	6

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727387.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	11	0	20

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727123.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	3	0	18

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727377.SJ.out.tab | grep 135858472 | grep 135858841
grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727508.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1	1	4	0	18

grep chr4 /mnt/lareaulab/cfbuenabadn/RNASeq/Mouse/ChenRNASeq/splicing/SJ_tables/SRR2727312.SJ.out.tab | grep 135858472 | grep 135858841
chr4	135858472	135858841	1	1

In [20]:
np.sum(pc)

6289

In [19]:
np.sum(nmd1)

3331

In [21]:
SJ_counts_table['Srsf10_nmdSE_1_I1'] = nmd1
SJ_counts_table['Srsf10_nmdSE_1_I2'] = nmd2
SJ_counts_table['Srsf10_nmdSE_1_SE'] = pc


In [25]:
SJ_counts_table.T.to_csv('SJ_counts.tab', sep='\t', index=True, header=True)